In [4]:
import pandas as pd
import yfinance as yf
from datetime import date
import pandas_ta as ta

In [49]:
def analyze_stock(stock, dados, start, end, window_size):
    # Baixa os dados históricos da ação do Yahoo Finance
    data = yf.download(stock, start=start, end=end)
    
    # Verifica se os dados foram baixados corretamente
    if not data.empty:
        # Reseta o índice para tratar a data como uma coluna
        data.reset_index(inplace=True)
        # Converte a coluna de data para o formato de string 'AAAA-MM-DD'
        dados['Date'] = data['Date'].dt.strftime('%Y-%m-%d')
        # Calcula a variação percentual diária do preço de fechamento ajustado
        dados['Var Percentual'] = data['Adj Close'].pct_change() * 100
        # Calcula a média móvel e o desvio padrão das variações percentuais
        media_rolling = dados['Var Percentual'].rolling(window=window_size).mean()
        desvio_padrao_rolling = dados['Var Percentual'].rolling(window=window_size).std()
        dados['Variancia'] =  dados['Var Percentual'].rolling(window=window_size).var()
        # Calculando a média da variância dos retornos usando a mesma janela
        dados['Media Variancia'] = dados['Variancia'].rolling(window=window_size).mean()
        if isinstance(data, pd.Series):
            data = data.to_frame()
        # Calculando o RSI
        rsi = data.ta.rsi(close='Adj Close', length=14)
        dados['rsi'] = rsi
        dados['Stock'] = stock
        print(type(dados['Date']))
        temp_datas = pd.to_datetime(dados['Date'])
        dados['AnoMes'] = temp_datas.dt.to_period('M')
        #print("Dataframe\n {}".format(dados))
        #dsv_sup1 = media_rolling + (desvio_padrao_rolling * 1.5)
        #dsv_sup2 = media_rolling + (desvio_padrao_rolling * 3)
        #dsv_inf1 = media_rolling - (desvio_padrao_rolling * 1.5)
        #dsv_inf2 = media_rolling - (desvio_padrao_rolling * 3)
        dados_filt = dados [(dados['rsi']>= 80) & (dados['Variancia'] < dados['Media Variancia']) & (dados['Var Percentual']< 4)] 
        dados_unique = dados_filt.drop_duplicates(subset='AnoMes', keep='first')
        dados_unique = dados_unique.drop('AnoMes',axis=1)
        print("Dados únicos")
        print(dados_unique)
        return dados_unique
    else:
        return dados

In [50]:
# Definição de parâmetros iniciais
stocksymbols = ["PETR4.SA", 'VALE3.SA', 'ITUB4.SA', 'BBAS3.SA', 'BBDC3.SA']
startdate = date(2002, 1, 1)
end_date = date(2023, 12, 31)
window_size = 100

# Processamento de múltiplas ações
all_results = pd.DataFrame()
dados = pd.DataFrame(columns=('Date','AnoMes','Var Percentual','rsi','Variancia','Media Variancia','Stock'))
for stock in stocksymbols:
    stock_results = analyze_stock(stock, dados, startdate, end_date, window_size)
    if not stock_results.empty:
        # Combina os resultados de todas as ações em um único DataFrame
        all_results = pd.concat([all_results, stock_results], ignore_index=True)

# Salva os resultados finais em um arquivo JSON
if not all_results.empty:
    all_results.to_json('filtered_vendacoberta_results.json', orient='records')
    print(all_results)
else:
    print("Nenhum dado foi encontrado para as condições especificadas.")

[*********************100%***********************]  1 of 1 completed
<class 'pandas.core.series.Series'>
Dados únicos
            Date  Var Percentual        rsi  Variancia  Media Variancia  \
712   2004-09-23        2.837153  81.101000   3.277354         4.746867   
718   2004-10-01        0.170518  84.565755   2.306182         4.577960   
961   2005-09-19        2.530859  82.595272   2.423114         2.936716   
1288  2006-12-27        2.305522  81.136591   2.410940         3.310577   
1290  2007-01-02        1.965523  84.692541   2.436940         3.288886   
1627  2008-05-19        3.842139  81.274123   7.882135        10.578871   
3716  2016-10-21        1.183763  80.122972   8.391731        15.635679   
4032  2018-01-22        1.150060  80.036968   3.100011         3.967187   
5366  2023-06-12        1.750327  81.942058   4.836659         7.173687   

         Stock  
712   PETR4.SA  
718   PETR4.SA  
961   PETR4.SA  
1288  PETR4.SA  
1290  PETR4.SA  
1627  PETR4.SA  
3716  PETR4.